# Boba tea shops in Denver Metro area

## 1.Introduction/Business Problem

Boba tea shops are becoming more popular during these years. Boba tea originates in Taiwan. They are unique and hard to find unlike coffee shops that you can see at almost every corner of the city. The more asian populations they city has, the more boba tea shops you can find.

Denver is one of the most growing cities in USA. More people are moving to Denver every year, realtor industry is growing, airlines industry is also growing. So if someone is looking to open a Boba tea shop Denver Metropolitan Area, where would it be the best area to open it? 

## 2.Data

2.1. from Foursquare, we will use Venue Search with paramters 

        -Latitue and Longtitude of Denver
        -radius
        -limit = 100
        -categoryId which is Bubble Tea Shop 52e81612bcbc57f1066b7a0c
    
to get data

        -venueid
        -venuename
        -latitude
        -longitude
        -city
        -postalcode
        -category_primaryid
        -category_primary
    

2.2. from Foursquare, we will al so use Venue Details which is a premium call to get

        -rating
        -price
        -hours
        -hours_popular
        -description
        -popullarity_score
    
2.3. Postal code, city, latitude, longtitude from https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/?refine.state=CO